To convert a depth image to a 3D point cloud, given the camera matrix K, you can follow these steps:

- Understand the Camera Matrix K:
    $$K= \begin{bmatrix} f_x & 0 & 0 \\ 0 & f_y & 0 \\ 0 & c_x & c_y \end{bmatrix} $$

- $f_x$ & $f_y$ Focal lengths in pixels along the x and y axes.
- $c_x$ and $c_y$ Principal point (optical center) coordinates in the image.

- Depth Image Representation:
The depth image is a 2D array where each pixel value represents the depth $z$
 - $z$ from the camera to the object at that pixel.

Back-Project Pixels to 3D Points:
For each pixel $(u,v)$
(u,v) in the depth image, the corresponding 3D point $(X,Y,Z)$ in the camera coordinate system can be obtained using the depth value $z$ at that pixel:

$$Z=depth(u,v)$$
 $$X = \frac{(u-c_x).Z}{f_x} $$

 $$ Y = \frac{(v-c_y).Z}{f_y}$$

In [ ]:
#Depth Image to point cloud Converssion

def depth_image_to_pointcloud(depth_image, K):
    """
    Convert a depth image to a 3D point cloud.
    
    Parameters:
    - depth_image: (H, W) PyTorch tensor containing depth values.
    - K: (3, 3) PyTorch tensor representing the camera matrix.
    
    Returns:
    - point_cloud: (H * W, 3) PyTorch tensor containing 3D points.
    """
    H, W = depth_image.shape
    fx, fy = K[0, 0], K[1, 1]
    cx, cy = K[0, 2], K[1, 2]

    # Generate a grid of (u, v) coordinates
    u = torch.arange(0, W).view(1, -1).expand(H, W)
    v = torch.arange(0, H).view(-1, 1).expand(H, W)

    # Get the corresponding depth values for each pixel
    Z = depth_image

    # Back-project to 3D space
    X = (u - cx) * Z / fx
    Y = (v - cy) * Z / fy

    # Stack to create a point cloud (H * W, 3)
    point_cloud = torch.stack((X, Y, Z), dim=-1).reshape(-1, 3)

    return point_cloud

# Example usage:
# Assume depth_image is a (H, W) tensor containing depth values
# Assume K is a (3, 3) tensor representing the camera matrix

# Define example depth image and camera matrix
H, W = 480, 640  # Image dimensions
depth_image = torch.rand(H, W) * 10  # Random depth image
K = torch.tensor([[500, 0, 320],
                  [0, 500, 240],
                  [0, 0, 1]], dtype=torch.float32)

# Convert depth image to point cloud
point_cloud = depth_image_to_pointcloud(depth_image, K)

print("Point cloud shape:", point_cloud.shape)
print(point_cloud)


In [ ]:
# Create Cynthetic planes in pytorch Tensors which are rothgonal to each other 

import torch

# Define the resolution of the planes
num_points = 100  # Number of points in each dimension

# Create a grid of points for the x-y plane (z = 0)
x = torch.linspace(-1, 1, num_points)
y = torch.linspace(-1, 1, num_points)
z_xy = torch.zeros(num_points)

# Create meshgrid for the x-y plane
xx, yy = torch.meshgrid(x, y, indexing="ij")
xy_plane = torch.stack([xx, yy, z_xy.expand_as(xx)], dim=-1).reshape(-1, 3)

# Create a grid of points for the y-z plane (x = 0)
z = torch.linspace(-1, 1, num_points)
y = torch.linspace(-1, 1, num_points)
x_yz = torch.zeros(num_points)

# Create meshgrid for the y-z plane
yy, zz = torch.meshgrid(y, z, indexing="ij")
yz_plane = torch.stack([x_yz.expand_as(yy), yy, zz], dim=-1).reshape(-1, 3)

# Combine the points from both planes into a single tensor
planes_tensor = torch.cat([xy_plane, yz_plane], dim=0)

print("Generated tensor shape:", planes_tensor.shape)
print(planes_tensor)


In [ ]:
#VIsualize those planes
import plotly.graph_objs as go
import plotly.io as pio

def visualise_tensor(point_cloud):
    # Extract x, y, z coordinates
    x = point_cloud[:, 0].numpy()
    y = point_cloud[:, 1].numpy()
    z = point_cloud[:, 2].numpy()

    # Create the 3D scatter plot using Plotly
    trace = go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=z,                # Set color to the z value to colorize points by depth
            colorscale='Viridis',   # Color scale
            opacity=0.8
        )
    )

    layout = go.Layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        margin=dict(l=0, r=0, b=0, t=0)  # Tight layout
    )
    fig = go.Figure(data=[trace], layout=layout)

    # Show the plot
    pio.show(fig)

In [ ]:

visualise_tensor(planes_tensor)

In [ ]:
## adding noise to plane_tensors 

# Add Gaussian noise to the planes
noise_std = 0.05  # Standard deviation of the noise
noise = torch.randn_like(planes_tensor) * noise_std
noisy_planes_tensor = planes_tensor + noise

In [ ]:
visualise_tensor(noisy_planes_tensor)

In [ ]:
import torch

def fit_plane(points):
    """
    Fits a plane to a set of 3D points using least squares.
    
    Parameters:
    - points: (N, 3) tensor containing the noisy 3D points.
    
    Returns:
    - plane_params: tensor [a, b, c, d] representing the plane equation ax + by + cz + d = 0
    """
    # Extract X, Y, Z coordinates
    X = points[:, 0]
    Y = points[:, 1]
    Z = points[:, 2]

    # Construct the A matrix and b vector for Ax + By + Cz + D = 0
    ones = torch.ones_like(X)
    A = torch.stack([X, Y, Z, ones], dim=1)  # (N, 4)
    
    # Fit using SVD: solving A * plane_params = 0
    U, S, Vt = torch.linalg.svd(A)
    plane_params = Vt[-1, :]  # Last row of Vt (corresponding to smallest singular value)

    # Normalize the plane parameters to make the plane equation unique
    plane_params /= plane_params.norm()
    
    return plane_params

# Example usage:
# Assuming noisy_planes_tensor is the noisy point cloud tensor generated earlier

# Fit a plane to the first plane points (first 10000 points)
plane1_params = fit_plane(noisy_planes_tensor[:10000])

# Fit a plane to the second plane points (last 10000 points)
plane2_params = fit_plane(noisy_planes_tensor[10000:])

print("Fitted plane 1 parameters: ", plane1_params)
print("Fitted plane 2 parameters: ", plane2_params)


$$ distance = \frac {ax+by+cz+d}{\sqrt{a^2+b^2+c^2} } $$

In [ ]:
import torch

def project_points_to_plane(points, plane_params):
    """
    Projects 3D points onto a fitted plane.

    Parameters:
    - points: (N, 3) tensor containing the noisy 3D points.
    - plane_params: tensor [a, b, c, d] representing the plane equation ax + by + cz + d = 0

    Returns:
    - corrected_points: (N, 3) tensor of points aligned to the plane.
    """
    # Extract plane parameters
    a, b, c, d = plane_params

    # Normal vector of the plane
    normal_vector = torch.tensor([a, b, c])

    # Calculate the distance of each point from the plane
    distances = (points @ normal_vector + d) / normal_vector.norm()

    # Correct the points by projecting onto the plane
    corrected_points = points - distances.unsqueeze(1) * normal_vector.unsqueeze(0) / normal_vector.norm()

    return corrected_points

# Example usage:
# Assuming noisy_planes_tensor is the noisy point cloud tensor generated earlier
# and plane1_params, plane2_params are the fitted plane parameters

# Correct the first plane points
corrected_plane1_points = project_points_to_plane(noisy_planes_tensor[:10000], plane1_params)

# Correct the second plane points
corrected_plane2_points = project_points_to_plane(noisy_planes_tensor[10000:], plane2_params)

# Combine corrected points into one tensor
corrected_planes_tensor = torch.cat([corrected_plane1_points, corrected_plane2_points], dim=0)

print("Corrected points shape:", corrected_planes_tensor.shape)
print(corrected_planes_tensor)


In [ ]:
visualise_tensor(corrected_planes_tensor)

In [ ]:
import torch

def calculate_mse(tensor1, tensor2):
    """
    Calculates the Mean Squared Error (MSE) between two tensors.

    Parameters:
    - tensor1: (N, 3) tensor representing ground truth points.
    - tensor2: (N, 3) tensor representing noisy or corrected points.

    Returns:
    - mse: Mean Squared Error between the two tensors.
    """
    return torch.mean((tensor1 - tensor2) ** 2)

# Assuming xy_plane and yz_plane are the original ground truth points
# generated before adding noise (as in the earlier code)

# Ground truth tensor (combination of xy_plane and yz_plane)
ground_truth_tensor = torch.cat([xy_plane, yz_plane], dim=0)

# Calculate the MSE between ground truth and noisy points
error_before_correction = calculate_mse(ground_truth_tensor, noisy_planes_tensor)

# Calculate the MSE between ground truth and corrected points
error_after_correction = calculate_mse(ground_truth_tensor, corrected_planes_tensor)

print("MSE before correction:", error_before_correction.item())
print("MSE after correction:", error_after_correction.item())


In [ ]:
error_after_correction

In [ ]:
error_before_correction

In [ ]:
import torch

def custom_huber_loss(residuals, delta=1.0):
    """
    Custom implementation of the Huber loss for residuals.
    
    Parameters:
    - residuals: Tensor of residuals.
    - delta: The threshold at which to switch from quadratic to linear loss.
    
    Returns:
    - loss: Tensor of Huber loss values.
    """
    abs_residuals = torch.abs(residuals)
    quadratic = torch.min(abs_residuals, torch.tensor(delta))
    linear = abs_residuals - quadratic
    loss = 0.5 * quadratic**2 + delta * linear
    return loss

def robust_plane_fitting(points, loss_fn, max_iters=100, lr=1e-3):
    """
    Perform robust plane fitting using a specified loss function.
    
    Parameters:
    - points: (N, 3) PyTorch tensor containing the 3D points.
    - loss_fn: The loss function to use (e.g., custom Huber loss).
    - max_iters: Maximum number of iterations for optimization.
    - lr: Learning rate for optimization.
    
    Returns:
    - plane_params: A tuple (a, b, c, d) representing the plane equation ax + by + cz + d = 0.
    """
    # Initialize plane parameters (a, b, c, d)
    plane_params = torch.randn(4, requires_grad=True)

    optimizer = torch.optim.Adam([plane_params], lr=lr)

    for _ in range(max_iters):
        optimizer.zero_grad()

        # Compute residuals: ax + by + cz + d
        a, b, c, d = plane_params
        residuals = points[:, 0] * a + points[:, 1] * b + points[:, 2] * c + d

        # Compute loss using the provided robust loss function
        loss = loss_fn(residuals).mean()

        # Backpropagate and update plane parameters
        loss.backward()
        optimizer.step()

    return plane_params.detach().numpy()

# Example usage:
# Generate some 3D points with outliers
N = 1000
points = torch.rand(N, 3) * 10
outliers = torch.rand(int(N * 0.1), 3) * 100  # Add 10% outliers
points = torch.cat([points, outliers], dim=0)

# Fit the plane using the custom Huber loss
plane_params = robust_plane_fitting(points, custom_huber_loss)

print("Fitted plane parameters (a, b, c, d):", plane_params)